In [9]:
import pandas as pd
import plotly.express as px
import datetime as dt
import humanize

In [14]:
def duration_to_string(x):
    if x!= x:
        return
    if x<0.000001:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='microseconds').replace('microsecond', 'µsec')
    if x < 1:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='milliseconds').replace('millisecond', 'msec')
    elif x < 60:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='seconds').replace('second', 'sec')
    else:
        return humanize.precisedelta(dt.timedelta(seconds=x), minimum_unit='minutes').replace('minute', 'min')


In [15]:
results_paths = ['network/report/results.json']

In [16]:
benches = pd.read_json(*results_paths, orient='records')
benches = benches.rename(columns={
    'backend': 'Backend Engine',
    'operation': 'Query',
    'dataset': 'Dataset',
    'iterations': 'Iterations',
    'dataset_size': 'Dataset Size, bytes',
    'seconds': 'Benchmark Duration, seconds',
})
benches['Iteration Duration, seconds'] = benches['Benchmark Duration, seconds'] / benches['Iterations']
benches['Iteration Duration'] = benches['Iteration Duration, seconds'].apply(duration_to_string)
benches.loc[benches['error'] != '', ['Iteration Duration, seconds', 'Iteration Duration']] = benches['Iteration Duration, seconds'].max(),'ERROR'
benches.loc[benches['error'] == 'NotImplementedError()', 'Iteration Duration'] = 'Not Implemented'
benches.loc[benches['error'] == 'TimeOut', 'Iteration Duration'] = 'TimeOut'


ValueError: cannot convert float NaN to integer

In [13]:
for name, group in benches.groupby(['Query']):
    if name == 'Close':
        continue

    # Docs:
    # https://plotly.com/python/bar-charts/
    fig = px.bar(
        group, 
        title=name,
        x='Dataset', 
        y='Iteration Duration, seconds',
        color='Backend Engine',
        barmode='group',
        text='Iteration Duration',
        log_y=True,
    )
    fig.update_layout(
        title_x=0.5,
        legend=dict(
            orientation='h',
            yanchor='bottom',
            y=1,
            x=0.5,
            xanchor='center',
        ),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=True),
        bargap=0.30,
        bargroupgap=0.10,
    )
    fig.update_traces(
        textfont_size=14,
        textangle=-60, 
        textposition='auto', 
        cliponaxis=False,
    )
    fig.show()